## **Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow.python.keras import backend as K
assert K.image_data_format() == 'channels_last'

In [0]:
# import tensorflow as tf
# %load_ext tensorboard

In [0]:
# %tensorboard --logdir '/content/drive/My Drive/dataset/FCN/graphs'

In [0]:
from glob import glob
from datetime import datetime
import numpy as np
import random
import cv2
import os
import re

import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 1.15.0


## **FCN_VGG**

In [0]:
class FCNVGG:
    # ---------------------------------------------------------------------------
    def __init__(self, session, num_classes):
        self.session = session
        self.num_classes = num_classes

    def build_from_metagraph(self, metagraph_file, checkpoint_file):
        self.__load_fcnvgg(metagraph_file, checkpoint_file)

    def build_from_vgg(self, vgg_path):
        self.__load_vgg(vgg_path)
        self.__make_result_tensors()

    def __load_fcnvgg(self, metagraph_file, checkpoint_file):
        saver = tf.train.import_meta_graph(metagraph_file)
        saver.restore(self.session, checkpoint_file)

        self.image_input = self.session.graph.get_tensor_by_name('image_input:0')
        self.keep_prob = self.session.graph.get_tensor_by_name('keep_prob:0')
        self.logits      = self.session.graph.get_tensor_by_name('sum/fcn_logits:0')
        self.softmax     = self.session.graph.get_tensor_by_name('result/fcn_softmax:0')
        self.classes     = self.session.graph.get_tensor_by_name('result/fcn_class:0')


    def __load_vgg(self, vgg_path):
        model = tf.saved_model.loader.load(self.session, ['vgg16'], vgg_path)
        graph = tf.get_default_graph()

        self.image_input = graph.get_tensor_by_name('image_input:0')
        self.keep_prob = graph.get_tensor_by_name('keep_prob:0')
        self.vgg_layer3 = graph.get_tensor_by_name('layer3_out:0')
        self.vgg_layer4 = graph.get_tensor_by_name('layer4_out:0')
        self.vgg_layer7 = graph.get_tensor_by_name('layer7_out:0')

    def __make_result_tensors(self):
        """
          :param correct_label:
          :param num_classes:
          :return:
        """
        # Use a shorter variable name for simplicity
        layer3, layer4, layer7 = self.vgg_layer3, self.vgg_layer4, self.vgg_layer7
        # Convert FCN to CONV
        # Apply 1x1 convolution in place of fully connected layer
        fcn8 = tf.layers.conv2d(layer7, filters=self.num_classes+1, kernel_size=1, name="fcn8")
        # Up-sample fcn8 with size depth=(4096?) to match size of layer 4
        # so that we can add skip connection with 4th layer
        fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
                                          kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
        # Add a skip connection between current final layer fcn8 and 4th layer
        fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
        # print(fcn9_skip_connected.shape)
        # Up-sample again
        fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
                                           kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")

        # Add skip connection
        fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")

        # Up-sample again
        # Final output: fcn11 = 8 * (4 * layer_out7 + 2 * layer_out4 + layer_out3)
        fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=self.num_classes+1,
                                           kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

        with tf.variable_scope('sum'):
            self.logits = tf.reshape(fcn11, (-1, self.num_classes), name="fcn_logits")

        with tf.name_scope('result'):
            self.softmax = tf.nn.softmax(self.logits, name="fcn_softmax")
            self.classes = tf.argmax(self.softmax, axis=-1, name="fcn_class")


    def get_optimizer(self, labels, learning_rate=0.0001):
        with tf.variable_scope('reshape'):
            labels_reshaped  = tf.reshape(labels, [-1, self.num_classes])
            logits_reshaped  = tf.reshape(self.logits, [-1, self.num_classes])

            losses          = tf.nn.softmax_cross_entropy_with_logits(
                                  labels=labels_reshaped,
                                  logits=logits_reshaped)
            loss            = tf.reduce_mean(losses)

        with tf.variable_scope('optimizer'):
            optimizer       = tf.train.AdamOptimizer(learning_rate)
            optimizer       = optimizer.minimize(loss)

        return optimizer, loss

## **Source_VOC**

In [0]:
import random
import cv2
import os
# from skimage.io import imshow
# import matplotlib.pyplot as plt
from glob import glob
import numpy as np


def color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return (byteval & (1 << idx)) != 0

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7 - j)
            g = g | (bitget(c, 1) << 7 - j)
            b = b | (bitget(c, 2) << 7 - j)
            c = c >> 3

        cmap[i] = np.array([b, g, r])

    cmap = cmap / 255 if normalized else cmap
    return cmap


def color_map_dict():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair',
              'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
              'train', 'tvmonitor', 'void']
    
    return dict(zip(labels, color_map()))


class VOCSource(object):
    def __init__(self):
        self.num_training = None
        self.num_validation = None
        self.train_generator = None
        self.valid_generator = None

        self.num_testing = None
        self.test_generator = None

        self.label_colors = color_map_dict()
        self.num_classes = 21   # ko bao gồm class void
        self.image_size = (512, 512)

    # -------------------------------------------------------------------------------
    def load_data(self, data_dir, images_txt, validation_size=None):
        images_dir = os.path.join(data_dir, 'VOC2012/JPEGImages/')   # very large
        labels_dir = os.path.join(data_dir, 'VOC2012/SegmentationClass/')  # 2913

        # train.txt: 1464 , val.txt: 1449 , Segmentation: 2913
        with open(images_txt, 'r') as f:
            image_names = f.readlines()
            image_paths, label_paths = [], {}
            for image in image_names:
                image_path = images_dir + image[:-1] + '.jpg'
                image_paths.append(image_path)
                label_paths[os.path.basename(image_path)] = labels_dir + image[:-1] + '.png'
        
        random.shuffle(image_paths)
        if validation_size is None:
            test_images = image_paths
            self.num_testing = len(test_images)
            self.test_generator = self.batch_generator(test_images, label_paths)
        else:
            num_images = len(image_paths)
            valid_images = image_paths[:int(validation_size * num_images)]
            train_images = image_paths[int(validation_size * num_images):]

            self.num_training = len(train_images)
            self.num_validation = len(valid_images)
            self.train_generator = self.batch_generator(train_images, label_paths)
            self.valid_generator = self.batch_generator(valid_images, label_paths)

    # -------------------------------------------------------------------------------
    def batch_generator(self, image_paths, label_paths):
        def gen_batch(batch_size):
            """
            :param batch_size:
            :return: generator of a batch contain (images, labels)
            """
            random.shuffle(image_paths)
            for offset in range(0, len(image_paths), batch_size):
                files = image_paths[offset:(offset + batch_size)]
                images = []
                labels = []
                for image_file in files:
                    label_file = label_paths[os.path.basename(image_file)]  # base name get file name from path
                    image = cv2.resize(cv2.imread(image_file), self.image_size)
                    label = cv2.resize(cv2.imread(label_file), self.image_size)

                    label_bg   = np.zeros([image.shape[0], image.shape[1]], dtype=bool)
                    label_list = []
                    for obj, color_bgr in self.label_colors.items():
                        if obj == 'background':
                            continue
                        label_current  = np.all(label == color_bgr, axis=2)
                        label_bg      |= label_current
                        label_list.append(label_current)

                    label_bg   = ~label_bg
                    label_all  = np.dstack([label_bg, *label_list])
                    label_all  = label_all.astype(np.float32)

                    images.append(image.astype(np.float32))
                    labels.append(label_all)

                yield np.array(images), np.array(labels)
        return gen_batch

## **Utils**

In [0]:
# from source_kitti import KittiSource
# from source_voc import VOCSource
# from fcnvgg import FCNVGG


def load_kitti_source():
    return KittiSource()


def load_voc_source():
    return VOCSource()


def load_fcnvgg(session, num_classes):
    return FCNVGG(session, num_classes)


# **Train**

In [0]:
# Killing optional CPU driver warnings
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# tf.logging.set_verbosity(tf.logging.ERROR)

# Specify all directory paths
data_dir = '/content/drive/My Drive/dataset/FCN/VOCdevkit/'
vgg_dir = '/content/drive/My Drive/dataset/FCN/vgg/'
log_dir = '/content/drive/My Drive/dataset/FCN/graphs/'
model_dir = '/content/drive/My Drive/dataset/FCN/saved_model/'

images_txt = '/content/drive/My Drive/dataset/FCN/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt'

saved_model_dir = '/content/drive/My Drive/dataset/FCN/saved_model/20200212-095201/'
checkpoint_file = os.path.join(saved_model_dir, 'checkpoint')

In [0]:
def main():
    source = load_voc_source()
    source.load_data(data_dir, images_txt, validation_size=0.2)
    train_generator = source.train_generator
    valid_generator = source.valid_generator

    epochs = 120
    batch_size = 12

    # Declare some placeholder will use when training
    correct_label = tf.placeholder(tf.float32, [None, source.image_size[1], 
                                                source.image_size[0], source.num_classes])
    training_loss = tf.placeholder(tf.float32)
    training_loss_summary_op = tf.summary.scalar('training_loss',
                                            training_loss)
    validation_loss = tf.placeholder(tf.float32)
    validation_loss_summary_op = tf.summary.scalar('validation_loss',
                                              validation_loss)

    session_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    with tf.Session(config=session_config) as session:
        # Create instance of FCNVGG model
        net = load_fcnvgg(session, source.num_classes)
        checkpoint_dir = os.path.join(model_dir, datetime.utcnow().strftime("%Y%m%d-%H%M%S"))
        
        net.build_from_vgg(vgg_dir)
        saver = tf.train.Saver(max_to_keep=10)
        print("Model build successful, starting train")

        optimizer_op, loss_op = net.get_optimizer(correct_label, learning_rate=0.0001)
        loss_summary = tf.summary.scalar("Loss", loss_op)

        session.run(tf.global_variables_initializer())
        for epoch in range(1, epochs+1, 1):
            print("STARTING EPOCH {} ...".format(epoch))
            # ---------------------------------- #
            # Training process
            training_loss_total = 0
            generator = train_generator(batch_size)
            for X_batch, gt_batch in generator:
                _, loss = session.run([optimizer_op, loss_op],
                                      feed_dict={net.image_input: X_batch, correct_label: gt_batch,
                                                 net.keep_prob: 0.5})

                training_loss_total += loss * X_batch.shape[0]
            training_loss_total /= source.num_training

            # ----------------------------------- #
            # Validation process
            valid_loss_total = 0
            generator = valid_generator(batch_size)
            for X_batch, gt_batch in generator:
                _, loss = session.run([optimizer_op, loss_op],
                                      feed_dict={net.image_input: X_batch, correct_label: gt_batch,
                                                 net.keep_prob: 1.})

                valid_loss_total += loss * X_batch.shape[0]
            valid_loss_total /= source.num_validation
            # ----------------------------------- #

            print("EPOCH {} ...".format(epoch))
            print("Training loss = {:.3f}".format(training_loss_total))
            print("Validation loss = {:.3f}".format(valid_loss_total))

            # ----------------------------------- #
            # Write loss summary
            feed = {validation_loss: valid_loss_total,
                    training_loss: training_loss_total}
            loss_summary = session.run([training_loss_summary_op,
                                        validation_loss_summary_op],
                                       feed_dict=feed)
            summary_op = tf.summary.merge([loss_summary[0], loss_summary[1]])
            # writer.add_summary(summary_op.eval(), epoch)

            if epoch % 40 == 0:
                checkpoint = checkpoint_dir + '/epoch{}.ckpt'.format(epoch)
                saver.save(session, checkpoint)
                print('Checkpoint saved:', checkpoint)
            # ----------------------------------- #

# **Main**

In [0]:
main()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/dataset/FCN/vgg/variables/variables
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Model build successful, starting train
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

STARTING EPOCH 1 ...
EPOCH 1 ...
Training loss = 9.749
Validation loss = 1.265
STARTING EPOCH 2 ...
EPOCH 2 ...
Training loss = 1.231
Validation loss = 1.185
STARTING EPOCH 3 ...
EPOCH 3 ...
Training loss = 1.